In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('ETH-USD.csv')
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-11-09,308.644989,329.451996,307.056000,320.884003,320.884003,893249984
1,2017-11-10,320.670990,324.717987,294.541992,299.252991,299.252991,885985984
2,2017-11-11,298.585999,319.453003,298.191986,314.681000,314.681000,842300992
3,2017-11-12,314.690002,319.153015,298.513000,307.907990,307.907990,1613479936
4,2017-11-13,307.024994,328.415009,307.024994,316.716003,316.716003,1041889984
...,...,...,...,...,...,...,...
1586,2022-03-14,2518.486328,2604.034424,2505.299316,2590.696045,2590.696045,11244398839
1587,2022-03-15,2590.668945,2662.329590,2515.765869,2620.149658,2620.149658,12861105614
1588,2022-03-16,2620.028564,2781.307129,2610.764404,2772.055664,2772.055664,17915109769
1589,2022-03-17,2771.964111,2826.160645,2751.560791,2814.854492,2814.854492,12685265194


In [3]:
#func to calculate exponential moving avg
def EMA(data,period=20,column='Close'):
    return data[column].ewm(span=period,adjust=False).mean()

In [4]:
#func to calculate Rellative Strength Index
def RSI(data,period=15,column='Close'):
    delta = data[column].diff(1)
    delta = delta.dropna()
    up = delta.copy()
    down = delta.copy()
    up[up<0]=0
    down[down>0]=0
    data['up'] = up
    data['down'] = down
    AVG_Gain = EMA(data,period,column='up')
    AVG_Loss = EMA(data,period,column='down')
    RS = AVG_Gain / AVG_Loss
    RSI = 100 - (100 / (1 + RS))
    
    data['RSI'+str(period)] = RSI
    return data

In [5]:
RSI(df,7)
RSI(df,14)
RSI(df,20)
df['EMA15'] = EMA(df,15)
df['EMA20'] = EMA(df,20)
df['EMA50'] = EMA(df,50)
df

,Date,Open,High,Low,Close,Adj Close,Volume,up,down,RSI7,RSI14,RSI20,EMA15,EMA20,EMA50
0,2017-11-09,308.644989,329.451996,307.056000,320.884003,320.884003,893249984,NaN,NaN,NaN,NaN,NaN,320.884003,320.884003,320.884003
1,2017-11-10,320.670990,324.717987,294.541992,299.252991,299.252991,885985984,0.000000,-21.631012,0.000000,0.000000,0.000000,318.180126,318.823907,320.035728
2,2017-11-11,298.585999,319.453003,298.191986,314.681000,314.681000,842300992,15.428009,0.000000,-31.189731,-12.325293,-8.117159,317.742736,318.429344,319.825739
3,2017-11-12,314.690002,319.153015,298.513000,307.907990,307.907990,1613479936,0.000000,-6.773010,-26.374600,-11.601003,-7.809572,316.513392,317.427310,319.358376
4,2017-11-13,307.024994,328.415009,307.024994,316.716003,316.716003,1041889984,8.808013,0.000000,-72.569709,-22.393386,-14.019790,316.538719,317.359567,319.254754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586,2022-03-14,2518.486328,2604.034424,2505.299316,2590.696045,2590.696045,11244398839,71.751465,0.000000,6392.081708,-588.837444,-502.260017,2638.285841,2665.266323,2829.976834
1587,2022-03-15,2590.668945,2662.329590,2515.765869,2620.149658,2620.149658,12861105614,29.453613,0.000000,438.273133,-2230.661171,-895.434770,2636.018818,2660.969497,2821.748317
1588,2022-03-16,2620.028564,2781.307129,2610.764404,2772.055664,2772.055664,17915109769,151.906006,0.000000,145.061494,276.781562,465.783505,2653.023424,2671.549132,2819.799585
1589,2022-03-17,2771.964111,2826.160645,2751.560791,2814.854492,2814.854492,12685265194,42.798828,0.000000,133.992801,230.973638,356.540520,2673.252307,2685.197262,2819.605660


In [6]:
#create target column to determine if tomorrow price will be greater than todays price then put 1 else 0
df['Target'] = np.where(df['Close'].shift(-1) > df['Close'],1,0)

In [7]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,up,down,RSI7,RSI14,RSI20,EMA15,EMA20,EMA50,Target
0,2017-11-09,308.644989,329.451996,307.056000,320.884003,320.884003,893249984,NaN,NaN,NaN,NaN,NaN,320.884003,320.884003,320.884003,0
1,2017-11-10,320.670990,324.717987,294.541992,299.252991,299.252991,885985984,0.000000,-21.631012,0.000000,0.000000,0.000000,318.180126,318.823907,320.035728,1
2,2017-11-11,298.585999,319.453003,298.191986,314.681000,314.681000,842300992,15.428009,0.000000,-31.189731,-12.325293,-8.117159,317.742736,318.429344,319.825739,0
3,2017-11-12,314.690002,319.153015,298.513000,307.907990,307.907990,1613479936,0.000000,-6.773010,-26.374600,-11.601003,-7.809572,316.513392,317.427310,319.358376,1
4,2017-11-13,307.024994,328.415009,307.024994,316.716003,316.716003,1041889984,8.808013,0.000000,-72.569709,-22.393386,-14.019790,316.538719,317.359567,319.254754,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586,2022-03-14,2518.486328,2604.034424,2505.299316,2590.696045,2590.696045,11244398839,71.751465,0.000000,6392.081708,-588.837444,-502.260017,2638.285841,2665.266323,2829.976834,1
1587,2022-03-15,2590.668945,2662.329590,2515.765869,2620.149658,2620.149658,12861105614,29.453613,0.000000,438.273133,-2230.661171,-895.434770,2636.018818,2660.969497,2821.748317,1
1588,2022-03-16,2620.028564,2781.307129,2610.764404,2772.055664,2772.055664,17915109769,151.906006,0.000000,145.061494,276.781562,465.783505,2653.023424,2671.549132,2819.799585,1
1589,2022-03-17,2771.964111,2826.160645,2751.560791,2814.854492,2814.854492,12685265194,42.798828,0.000000,133.992801,230.973638,356.540520,2673.252307,2685.197262,2819.605660,0


In [8]:
df = df[1:]
df

,Date,Open,High,Low,Close,Adj Close,Volume,up,down,RSI7,RSI14,RSI20,EMA15,EMA20,EMA50,Target
1,2017-11-10,320.670990,324.717987,294.541992,299.252991,299.252991,885985984,0.000000,-21.631012,0.000000,0.000000,0.000000,318.180126,318.823907,320.035728,1
2,2017-11-11,298.585999,319.453003,298.191986,314.681000,314.681000,842300992,15.428009,0.000000,-31.189731,-12.325293,-8.117159,317.742736,318.429344,319.825739,0
3,2017-11-12,314.690002,319.153015,298.513000,307.907990,307.907990,1613479936,0.000000,-6.773010,-26.374600,-11.601003,-7.809572,316.513392,317.427310,319.358376,1
4,2017-11-13,307.024994,328.415009,307.024994,316.716003,316.716003,1041889984,8.808013,0.000000,-72.569709,-22.393386,-14.019790,316.538719,317.359567,319.254754,1
5,2017-11-14,316.763000,340.177002,316.763000,337.631012,337.631012,1069680000,20.915009,0.000000,1196.904894,-66.512132,-34.327466,319.175255,319.290181,319.975391,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586,2022-03-14,2518.486328,2604.034424,2505.299316,2590.696045,2590.696045,11244398839,71.751465,0.000000,6392.081708,-588.837444,-502.260017,2638.285841,2665.266323,2829.976834,1
1587,2022-03-15,2590.668945,2662.329590,2515.765869,2620.149658,2620.149658,12861105614,29.453613,0.000000,438.273133,-2230.661171,-895.434770,2636.018818,2660.969497,2821.748317,1
1588,2022-03-16,2620.028564,2781.307129,2610.764404,2772.055664,2772.055664,17915109769,151.906006,0.000000,145.061494,276.781562,465.783505,2653.023424,2671.549132,2819.799585,1
1589,2022-03-17,2771.964111,2826.160645,2751.560791,2814.854492,2814.854492,12685265194,42.798828,0.000000,133.992801,230.973638,356.540520,2673.252307,2685.197262,2819.605660,0


In [10]:
#list of columns to keep
keep_columns = df.drop(['Date','High','Low','Open','Volume','Adj Close','up','down','Close','Target'],axis=1).columns
X = df[keep_columns].values
Y = df['Target'].values

In [15]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [24]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=1,criterion='entropy',random_state=1)
forest.fit(X_train,Y_train)

RandomForestClassifier(criterion='entropy', n_estimators=1, random_state=1)

In [25]:
forest.score(X_test,Y_test)

0.46855345911949686

In [26]:
#models predicted val
forest_predictions = forest.predict(X_test)
forest_predictions

array([1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0,

In [27]:
Y_test

array([0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,